# Text-Based Industry Classification

## Setting up environment

In [68]:
import pandas as pd
import numpy as np
import math
import os
import pickle
from collections import defaultdict
os.chdir("/Users/hanyi.wang/Documents/Projects/Text-based Correlation Model/Data")

### Understanding the Data

 **First we examine the data with the index file:**

In [69]:
with open('index_file.dat', 'rb') as index:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data_index = pickle.load(index)

with open('Business Description.dat', 'rb') as d:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data_businessD = pickle.load(d)
    
data_index.head()


,CIK,Comname,File_Type,File_Date,txt_path,html_path,txt_name
631,1000298,IMPAC MORTGAGE HOLDINGS INC,10-K,2017-03-09,edgar/data/1000298/0001558370-17-001542.txt,edgar/data/1000298/0001558370-17-001542-index....,10002980001558370-17-001542.txt
652,1000623,SCHWEITZER MAUDUIT INTERNATIONAL INC,10-K,2017-02-24,edgar/data/1000623/0001000623-17-000036.txt,edgar/data/1000623/0001000623-17-000036-index....,10006230001000623-17-000036.txt
681,1000683,BLONDER TONGUE LABORATORIES INC,10-K,2017-03-31,edgar/data/1000683/0001144204-17-017902.txt,edgar/data/1000683/0001144204-17-017902-index....,10006830001144204-17-017902.txt
686,1000694,NOVAVAX INC,10-K,2017-02-27,edgar/data/1000694/0001144204-17-011334.txt,edgar/data/1000694/0001144204-17-011334-index....,10006940001144204-17-011334.txt
708,1000697,WATERS CORP /DE/,10-K,2017-02-24,edgar/data/1000697/0001193125-17-056239.txt,edgar/data/1000697/0001193125-17-056239-index....,10006970001193125-17-056239.txt


In [44]:
data_index = data_index[data_index['txt_path'].notna()]
print(data_index['File_Date'].dtypes)
data_index['File_Date'] = pd.to_datetime(data_index['File_Date'],format = '%Y-%m-%d')
data_index['year'] = data_index['File_Date'].dt.year

#seperate years individually
data_2016 = data_index[data_index['year']==2016]
data_2017 = data_index[data_index['year']==2017]
data_2018 = data_index[data_index['year']==2018]
data_2019 = data_index[data_index['year']==2019]

data = data_2018 # change according to year

datetime64[ns]


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2185 entries, 93 to 197782
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   CIK        2185 non-null   int64         
 1   Comname    2185 non-null   object        
 2   File_Type  2185 non-null   object        
 3   File_Date  2185 non-null   datetime64[ns]
 4   txt_path   2185 non-null   object        
 5   html_path  2185 non-null   object        
 6   txt_name   2185 non-null   object        
 7   year       2185 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 153.6+ KB


In [46]:
data_index.head()

,CIK,Comname,File_Type,File_Date,txt_path,html_path,txt_name,year
631,1000298,IMPAC MORTGAGE HOLDINGS INC,10-K,2017-03-09,edgar/data/1000298/0001558370-17-001542.txt,edgar/data/1000298/0001558370-17-001542-index....,10002980001558370-17-001542.txt,2017
652,1000623,SCHWEITZER MAUDUIT INTERNATIONAL INC,10-K,2017-02-24,edgar/data/1000623/0001000623-17-000036.txt,edgar/data/1000623/0001000623-17-000036-index....,10006230001000623-17-000036.txt,2017
681,1000683,BLONDER TONGUE LABORATORIES INC,10-K,2017-03-31,edgar/data/1000683/0001144204-17-017902.txt,edgar/data/1000683/0001144204-17-017902-index....,10006830001144204-17-017902.txt,2017
686,1000694,NOVAVAX INC,10-K,2017-02-27,edgar/data/1000694/0001144204-17-011334.txt,edgar/data/1000694/0001144204-17-011334-index....,10006940001144204-17-011334.txt,2017
708,1000697,WATERS CORP /DE/,10-K,2017-02-24,edgar/data/1000697/0001193125-17-056239.txt,edgar/data/1000697/0001193125-17-056239-index....,10006970001193125-17-056239.txt,2017


In [59]:
num_firm = data.count()[0]
num_firm

2185

### Now the Business Description Data:

In [48]:
data_businessD['10339050001033905-18-000014.txt'][0:1000]
#data_businessD[data_index.iat[i, 6]]

'ITEM 1. BUSINESS Overview We develop, manufacture and sell proprietary biological testing technologies and products with applications throughout the life sciences industries, including diagnostics, pharmaceutical and research. These industries depend on a broad range of tests, called assays, to perform diagnostic testing and conduct life science research. We have established a position in several segments of the life sciences industries by developing and delivering products that satisfy a variety of customer needs in specific market segments, including multiplexing, accuracy, precision, sensitivity, specificity, reduction of labor and ability to test for proteins and nucleic acids. These needs are addressed by our proprietary technologies. Multiplexing, the foundation of our Company, allows the end user in a laboratory to generate multiple laboratory results from a single sample with a single assay. This is important because our end user customers, which include laboratory professiona

## Tokenization

In [49]:
import nltk
from nltk.book import *

### Tokenize BD and Select Noun(NN) and Propernoun(NNP)

In [52]:
#tokens = nltk.word_tokenize(data_businessD['10339050001033905-18-000014.txt'][0:1000])
import enchant

all_noun = []
all_propernoun = []

for i in range(0,len(data)): #len(data_index)
    tokens = nltk.word_tokenize(data_businessD[data.iat[i, 6]])
    
    #Using Enchant package to standerdize our text input
    legal_dict = enchant.Dict("en_US")
    tokens= [word for word in tokens if (word.isalpha()) & (legal_dict.check(word))]
    
    #nltk Part Of Speech Package to select NN and NNP.
    pos = nltk.pos_tag(tokens)
    word_list = set([word for word in pos if (word[1] == 'NN') | (word[1] == 'NNP')])
    dic_noun = [s[0] for s in word_list if s[1] == 'NN']
    all_noun.append(dic_noun)
    dic_propernoun = [s[0] for s in word_list if s[1] == 'NNP']
    all_propernoun.append(dic_propernoun)
    




[See reference for POS taggings](https://www.guru99.com/pos-tagging-chunking-nltk.html)

In [60]:
all_noun_dic = defaultdict(int)
all_propernoun_dic = defaultdict(int)
# Bring 'NN' into our dictionary
for i in range(0,len(data)):
    for j in range(len(all_noun[i])):
        all_noun_dic[all_noun[i][j]] += 1

# Bring 'NNP' into our dictionary
for i in range(len(data)):
    for j in range(len(all_propernoun[i])):
        all_propernoun_dic[all_propernoun[i][j]] += 1

   ### Screening

In [61]:
# Save NNPs that have 90%+ higher case ratio
all_propernoun_selected_dic = {i:all_propernoun_dic[i] for i in all_propernoun_dic if all_propernoun_dic[i] / (all_propernoun_dic[i] + all_noun_dic[i.lower()]) > 0.9}

In [62]:
# Keep only Text with less than 25% recurrence ratio to screen out common terms
num = data.count()
all_dic = {**all_noun_dic, **defaultdict(int,all_propernoun_selected_dic)}
all_selected_word = [i for i in all_dic if (all_dic[i] / num[0] < 0.25) & (all_dic[i]) != 0]

## Construct Token Mappings

In [74]:
matrix = []
for i in range(len(data)):
    vector = [(word in all_noun[i]) | (word in all_propernoun[i]) for word in all_selected_word]
    matrix.append(vector)
df = pd.DataFrame(matrix, index = data['Comname'].tolist(), columns = all_selected_word).replace({True:1, False:0})  
df.to_pickle("./Mappings-2018.pkl") #Write mappings to pickel

In [64]:
nulls = df.isna().sum()/df.shape[0]
df.info()
df.head()
# ap = df[df.loc[['POLARITYTE, INC.']]>0]
# print(df.loc[['POLARITYTE, INC.']]>0)

<class 'pandas.core.frame.DataFrame'>
Index: 2185 entries, HENRY SCHEIN INC to WASHINGTON FEDERAL INC
Columns: 17181 entries, track to Hermitage
dtypes: int64(17181)
memory usage: 286.4+ MB


,track,mandate,shift,announcement,defendant,incurring,payroll,alternate,disproportionate,speculation,...,Bombshells,SEVILLE,Jaguars,ONYX,Excepting,SALOON,TOOTSIES,INTELSAT,Nettles,Hermitage
HENRY SCHEIN INC,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
WATERS CORP /DE/,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"SANGAMO THERAPEUTICS, INC",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"TG THERAPEUTICS, INC.",1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
NORTHWEST PIPE CO,0,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


## Construct Correlation Matrix

In [75]:
# some normalization 
norm = np.linalg.norm(df, axis = 1)
norm = norm.reshape(norm.shape[0],1)
df_norm = df/norm[:, ]


In [83]:
cor_matrix = []
cor_matrix = np.matrix(df_norm).dot(np.matrix(df_norm).T)
cor_df = pd.DataFrame(cor_matrix,index = data['Comname'].tolist(),columns = data['Comname'].tolist())
cor_df.head()
cor_df.to_pickle('2018_correlation.pkl')

In [ ]:
# fin_matrix = []
# for i in range(len(df)):
#     fin_vector = []
#     for j in range(len(df)):
#         dot = np.matrix(df.iloc[i]).dot(np.matrix(df.iloc[j]).T)
#         leni = math.sqrt(np.matrix(df.iloc[i]).dot(np.matrix(df.iloc[i]).T))
#         lenj = math.sqrt(np.matrix(df.iloc[j]).dot(np.matrix(df.iloc[j]).T))
#         fin_vector.append(dot/(leni * lenj))
#     fin_matrix.append(fin_vector)
# fin_df = pd.DataFrame(fin_matrix, index = index['Comname'].tolist(), columns = index['Comname'].tolist())
# din_df.to_pickle('2018.pkl')